Le Dubincc est calculé à partir d'un point, une orientation et un rayon de départ et d'arrivée

In [ ]:
%matplotlib notebook

import math
import sympy

import geometrik.threed as g3d
import goto.globe


from goto.globe.plot import GlobePlotMpl

In [ ]:
A_lat, A_lon, A_hdg, A_rad, Aw = 0.0, -10.0, math.radians(45.0), 2000000 / goto.globe.earth_radius, 1
B_lat, B_lon, B_hdg, B_rad, Bw = 20.0, 30.0, math.radians(60.0), 1500000 / goto.globe.earth_radius, -1

In [ ]:
A = goto.globe.Blip(A_lat, A_lon).as_vector # initial position
A_x, A_y = A.oriented_frame(A_hdg)
A_right = A.deflect(A_y, A_rad)
A_left = A.deflect(A_y, -A_rad)
C = A_right if Aw > 0 else A_left # center of the initial circle

In [ ]:
# C_x, C_y form a frame in which the projection of A is at angle 0 and
# any other projection has an angle in [0;360], increasing in the way of rotation
C_y = Aw * (A @ C).normalized()
C_x = A @ C_y

In [ ]:
B = goto.globe.Blip(B_lat, B_lon).as_vector # final position
B_x, B_y = B.oriented_frame(B_hdg)
B_right = B.deflect(B_y, B_rad)
B_left = B.deflect(B_y, -B_rad)
D = B_right if Bw > 0 else B_left # center of the final circle

In [ ]:
D_y = Bw * (B @ D).normalized()
D_x = -B @ D_y

In [ ]:
CD_dist = C.angle_to(D)
CD_dist

In [ ]:
# N_z is a vector perpendicular to the line which passes through both centers
N_z = (C @ D).normalized()
N_x = C
N_y = N_z @ N_x

In [ ]:
theta_1 = math.asin(
    (math.sin(A_rad)/math.sin(CD_dist) - math.sin(B_rad)/math.tan(CD_dist)) /
    math.sqrt(
        (-math.sin(A_rad) + math.sin(B_rad)*math.cos(CD_dist))**2 /
        (math.sin(A_rad)**2 - 2*math.sin(A_rad)*math.sin(B_rad)*math.cos(CD_dist) + math.sin(B_rad)**2)
    )
)


In [ ]:
theta_2 = math.asin(
    (math.sin(A_rad)/math.sin(CD_dist) + math.sin(B_rad)/math.tan(CD_dist)) / 
    math.sqrt(
        (math.sin(A_rad) + math.sin(B_rad)*math.cos(CD_dist))**2 /
        (math.sin(A_rad)**2 + 2*math.sin(A_rad)*math.sin(B_rad)*math.cos(CD_dist) + math.sin(B_rad)**2)
    )
)


In [ ]:
A_psi = math.asin(math.sin(A_rad) / math.sin(theta_1))
M_x = N_x.deflect(N_y, A_psi)
M_z = N_z.deflect(-M_x, theta_1)

In [ ]:
with GlobePlotMpl() as gpl :
    gpl.add_point(A, 'A', 'r')
    #gpl.add_point(A_y, 'Ay', 'r')
    gpl.add_point(B, 'B', 'b')
    #gpl.add_point(B_y, 'By', 'b')
    gpl.add_line(C, D, 'g')
    gpl.add_point(C_x, 'Cx', 'orange')
    gpl.add_point(C_y, 'Cy', 'orange')
    gpl.add_point(D_x, 'Dx', 'purple')
    gpl.add_point(D_y, 'Dy', 'purple')

    #gpl.add_point(Ny.subs(m_val), 'Ny', 'g')
    #gpl.add_point(Nz.subs(m_val), 'Nz', 'b')
    #gpl.add_great_circle(Nz.subs(m_val), 'k')
    if Aw > 0 :
        gpl.add_circle(A_right, A_rad, 'orange')
    else :
        gpl.add_circle(A_left, A_rad, 'orange')
    if Bw > 0 :
        gpl.add_circle(B_right, B_rad, 'purple')
    else :
        gpl.add_circle(B_left, B_rad, 'purple')

    gpl.add_great_circle(M_z, 'red')
    gpl.add_point(N_x, 'Nx', 'red')
    gpl.add_point(N_z, 'Nz', 'red')
    gpl.add_point(M_x, 'Mx', 'cyan')
    gpl.add_point(M_z, 'Mz', 'cyan')   

## return to horizontal

In [ ]:
theta, psi_a, psi_b, Ra, Rb, alpha = sympy.symbols('theta psi_a psi_b R_a R_b alpha')
Pa, Pb = sympy.symbols('P_a P_b')

In [ ]:
Nz = g3d.Vector(0, -sympy.sin(theta), sympy.cos(theta), is_symbolic=True)
Ny = g3d.Vector(0, sympy.cos(theta), sympy.sin(theta), is_symbolic=True)
Nx = g3d.Vector(1, 0, 0, is_symbolic=True)

In [ ]:
Ua = g3d.Vector(sympy.cos(psi_a), sympy.sin(psi_a), 0)
Ub = g3d.Vector(sympy.cos(psi_b), sympy.sin(psi_b), 0)

In [ ]:
theta_1 = sympy.asin(
    ((Pb**2*sympy.sin(alpha)**2 + Pa**2 - Pb**2) / (
        sympy.sin(alpha)*(Pa + Pb*sympy.cos(alpha))* sympy.sqrt(
            (Pa - Pb*sympy.cos(alpha))**2 /
            (Pa**2 - 2*sympy.cos(alpha)*Pa*Pb + Pb**2)
        )
    )).simplify()
).subs({'P_a': sympy.sin(Ra), 'P_b': sympy.sin(Rb)}).simplify()
theta_1

In [ ]:
theta_2 = sympy.asin(
    ((Pb**2*sympy.sin(alpha)**2 + Pa**2 - Pb**2) / (
        sympy.sin(alpha)*(Pa - Pb*sympy.cos(alpha))* sympy.sqrt(
            (Pa + Pb*sympy.cos(alpha))**2 /
            (Pa**2 + 2*sympy.cos(alpha)*Pa*Pb + Pb**2)
        )
    )).simplify()
).subs({'P_a': sympy.sin(Ra), 'P_b': sympy.sin(Rb)}).simplify()
print(theta_2)

In [ ]:
m_val = {'R_a': A_rad, 'R_b': B_rad, 'alpha': CD_dist}
m_val

In [ ]:
m_val = {'R_a': 0.32, 'R_b': 0.23, 'alpha': 0.85}
m_val

In [ ]:
with GlobePlotMpl() as gpl :
    m_val['theta'] = float(- theta_1.subs(m_val))
    gpl.add_point(Nx.subs(m_val), 'Nx', 'r')
    gpl.add_point(Ny.subs(m_val), 'Ny', 'g')
    gpl.add_point(Nz.subs(m_val), 'Nz', 'b')
    gpl.add_great_circle(Nz.subs(m_val), 'cyan')
    m_val['psi_a'] = float(sympy.asin(sympy.sin(Ra) / sympy.sin(theta)).subs(m_val))
    m_val['psi_b'] = float(sympy.asin(sympy.sin(Rb) / sympy.sin(theta)).subs(m_val))
    gpl.add_circle(Ua.subs(m_val), float(Ra.subs(m_val)), 'orange')
    gpl.add_point(Ua.subs(m_val), 'Ua', 'orange')
    gpl.add_point(Ub.subs(m_val), 'Ub', 'red')
    gpl.add_circle(Ub.subs(m_val), float(Rb.subs(m_val)), 'red')
    gpl.add_line(Ua.subs(m_val), Ub.subs(m_val), 'purple')